## Imports

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import warnings
warnings.filterwarnings(action='once')
data_path = str(Path(os.getcwd()).parent.absolute())+"/data"
figures_path = str(Path(os.getcwd()).parent.absolute())+"/reports/figures"

## Functions

In [ ]:
def industry_value_weighted(df, threshold, score, industries_sel, industries):
    factor = 0
    mkt_cap = df.loc[:,['LME']].sum()[0]
    for industry in industries:
        mkt_cap_industry = df.loc[df[industries_sel] == industry,['LME']].sum()[0]
        factor += top_minus_bottom_value_weighted(df.loc[df[industries_sel] == industry], threshold, score) * mkt_cap_industry
    return(factor / mkt_cap)

In [ ]:
def industry_weighted(df, threshold, score, industries_sel, industries):
    factor = 0
    mkt_cap = df.loc[:,['LME']].sum()[0]
    for industry in industries:
        mkt_cap_industry = df.loc[df[industries_sel] == industry,['LME']].sum()[0]
        factor += top_minus_bottom(df.loc[df[industries_sel] == industry], threshold, score) * mkt_cap_industry
    return(factor / mkt_cap)

In [ ]:
def top_minus_bottom_value_weighted(df, threshold, score):
    df_with_data = df.loc[df[score].isna() == False,:]
    threshold_value_best = df_with_data.loc[:,[score]].quantile(1-threshold, numeric_only=True)[0]
    threshold_value_worst = df_with_data.loc[:,[score]].quantile(threshold, numeric_only=True)[0]
    df_best = df_with_data.loc[df_with_data[score] >= threshold_value_best,['ExRet','LME',score]]
    df_worst = df_with_data.loc[df_with_data[score] <= threshold_value_worst,['ExRet','LME',score]]
    mkt_cap_best = df_best['LME'].sum()
    mkt_cap_worst = df_worst['LME'].sum()
    ExRet_best = (df_best['ExRet']*df_best['LME']).sum() / mkt_cap_best
    ExRet_worst = (df_worst['ExRet']*df_worst['LME']).sum() / mkt_cap_worst
    return(ExRet_best - ExRet_worst)

In [ ]:
def top_minus_bottom(df, threshold, score):
    df_with_data = df.loc[df[score].isna() == False,:]
    threshold_value_best = df_with_data.loc[:,[score]].quantile(1-threshold, numeric_only=True)[0]
    threshold_value_worst = df_with_data.loc[:,[score]].quantile(threshold, numeric_only=True)[0]
    df_best = df_with_data.loc[df_with_data[score] >= threshold_value_best,['ExRet','LME',score]]
    df_worst = df_with_data.loc[df_with_data[score] <= threshold_value_worst,['ExRet','LME',score]]
    nb_best = len(df_best)
    nb_worst = len(df_worst)
    ExRet_best = (df_best['ExRet']).sum() / nb_best
    ExRet_worst = (df_worst['ExRet']).sum() / nb_worst
    return(ExRet_best - ExRet_worst)

## Creating the portfolios

In [ ]:
# Reading raw data
raw = pd.read_csv("STOCK_SECURITIES_no_drops.csv")
raw = raw.loc[:,['date','GVKEY','ExRet','LME','ff17', 'INDUSTRY_ADJUSTED_SCORE','WEIGHTED_AVERAGE_SCORE', 'ENVIRONMENTAL_PILLAR_SCORE', 'SOCIAL_PILLAR_SCORE', 'GOVERNANCE_PILLAR_SCORE', 'CARBON_EMISSIONS_SCORE']]
raw = raw.rename(columns={'IVA_COMPANY_RATING':'RATING',
                    'INDUSTRY_ADJUSTED_SCORE':'ESG_ADJ',
                    'WEIGHTED_AVERAGE_SCORE':'ESG',
                    'ENVIRONMENTAL_PILLAR_SCORE':'E',
                    'SOCIAL_PILLAR_SCORE':'S',
                    'GOVERNANCE_PILLAR_SCORE':'G',
                    'CARBON_EMISSIONS_SCORE':'CO2'})

df = raw.set_index(['date', 'GVKEY'])

In [ ]:
# Creating flags for big and small portfolio for using bivariate portfolio and following Fama-French methodology

df['BIG']=False
big = []
for date in raw['date'].unique():
    # are we in june ? or in the first date ?
    if date[5:7] == '06' or len(big) == 0:
        df_single_date = df.loc[date].sort_values(by='LME', ascending=False)
        mkt_cap_90 = df_single_date.loc[:,['LME']].sum()[0]*0.9
        temp = 0
        for i,index in enumerate(df_single_date.index):
            if temp + df_single_date.loc[index,'LME'] < mkt_cap_90:
                temp += df_single_date.loc[index,'LME']
            else:
                big = df_single_date.index[0:i+1]
                break
    df.loc[(date,big),'BIG'] = True




In [ ]:
# Chosing the tresholds for the best and the worst, choosing which MSCI characteristics to use, and choosing the definition of industry (how many)
threshold = 0.3
scores = ['ESG_ADJ', 'ESG', 'E', 'S', 'G', 'CO2']
industries_sel = 'ff17'
industries = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0]

In [ ]:
# Following Fama-French methodology for factor creation
factors = pd.DataFrame()
for date in raw['date'].unique():
    df_big = df.loc[date][df.loc[date,'BIG']]
    df_small = df.loc[date][df.loc[date,'BIG']]
    for score in scores:
        enough_data = len(df.loc[date]) - df.loc[date][score].isna().sum() >=500 # more than 500 companies with data
        if enough_data:
            if score != 'ESG_ADJ':
                factors.loc[date,[score+'_V_IND_W']] = .5 * (industry_value_weighted(df_big, threshold, score, industries_sel, industries)
                                                        + industry_value_weighted(df_small, threshold, score, industries_sel, industries))

            else:     # For already made industry weighted score :
                factors.loc[date,[score]] = top_minus_bottom_value_weighted(df.loc[date], threshold, 'ESG_ADJ')
        else :
            if score != 'ESG_ADJ':
                factors.loc[date,[score+'_V_IND_W']] = np.nan
            else:
                factors.loc[date,[score]] = np.nan





In [ ]:
# Saving
factors.to_csv("green_factors.csv")